列举一些常见的数据加载和处理工具

- `torch.utils.data.Dataset`: 数据集的抽象类，需要自定义并实现 `__len__`（数据集大小）和 `__getitem__`（按索引获取样本）。
- `torch.utils.data.TensorDataset`: 基于张量的数据集，适合处理数据-标签对，直接支持批处理和迭代。
- `torch.utils.data.DataLoader`: 封装 Dataset 的迭代器，提供批处理、数据打乱、多线程加载等功能，便于数据输入模型训练。
- `torchvision.datasets.ImageFolder`: 从文件夹加载图像数据，每个子文件夹代表一个类别，适用于图像分类任务。

## pytorch内置数据集

- **MNIST**：手写数字图像数据集，用于图像分类任务。
- **CIFAR**：包含 10 个类别，60000 张 32x32 的彩色图像数据集，用于图像分类任务。
- **COCO**：通用物体检测、分割、关键点检测数据集，包含超过 330k 个图像和 2.5M 个目标实例的大规模数据集。
- **ImageNet**：包含超过 1400 万张图像，用于图像分类和物体检测等任务。
- **STL-10**：包含 100k 张 96x96 的彩色图像数据集，用于图像分类任务。
- **Cityscapes**：包含 5000 张精细注释的城市街道场景图像，用于语义分割任务。
- **SQUAD**：用于机器阅读理解任务的数据集。

以上数据集可以通过 `torchvision.datasets` 模块中的函数进行加载，也可以通过自定义的方式加载其他数据集。

下面详细讲解两个类

### torch.utils.data.Dataset

Dataset是一个抽象类

自定义数据集需要继承 `torch.utils.data.Dataset` 并重写以下两个方法：

- **`__len__`**：返回数据集的大小。
- **`__getitem__`**：按索引获取一个数据样本及其标签。

In [5]:
# 实例
import torch
from torch.utils.data import DataLoader

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label
    
# 生成实例数据
data = torch.randn(100, 5) # 100个样本，每个样本5个特征
# 注意这里样本标签为0和1，是因为这个区间是左闭右开的，所以取不到2
labels = torch.randint(0, 2, (100,)) # 100个样本，每个样本的标签为0或1

# 实例化数据集
dataset = MyDataset(data, labels)

# 测试数据集
print('数据集大小：', len(dataset))
print('第一个样本：', dataset[0])


数据集大小： 100
第一个样本： (tensor([-0.5490, -1.0026, -1.2437,  0.0412,  0.8429]), tensor(0))


### torch.utils.data.DataLoader

**DataLoader** 是 PyTorch 提供的数据加载器，用于批量加载数据集。

提供了以下功能：

- **批量加载**：通过设置 `batch_size`。
- **数据打乱**：通过设置 `shuffle=True`。
- **多线程加速**：通过设置 `num_workers`。
- **迭代访问**：方便地按批次访问数据。

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# 自定义数据集
class MyDataset(Dataset):
    def __init__(self, data, labels):
        # 数据初始化
        self.data = data
        self.labels = labels

    def __len__(self):
        # 返回数据集大小
        return len(self.data)

    def __getitem__(self, idx):
        # 按索引返回数据和标签
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label

# 生成示例数据
data = torch.randn(100, 5)  # 100 个样本，每个样本有 5 个特征
labels = torch.randint(0, 2, (100,))  # 100 个标签，取值为 0 或 1

# 实例化数据集
dataset = MyDataset(data, labels)
# 实例化 DataLoader
dataloader = DataLoader(dataset, batch_size=10, shuffle=True, num_workers=0)

# 遍历 DataLoader
for batch_idx, (batch_data, batch_labels) in enumerate(dataloader): #enumberate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标
    print(f"批次 {batch_idx + 1}")
    print("数据:", batch_data)
    print("标签:", batch_labels)
    if batch_idx == 2:  # 仅显示前 3 个批次
        break


# 自己的理解：先生成一批样本data，这批样本有100个，每个样本有5个特征
# 然后生成这批样本的标签，100个样本的标签为0或1
# 然后实例化数据集，是自定义的一个类MyDataset，这个类继承了torch.utils.data.Dataset
# 然后通过MyDataset类实例化一个数据集dataset，也就是将data和labels传入MyDataset类中，放到dataset中
# 然后实例化DataLoader, 也就是将dataset传入DataLoader中，并设置batch_size=10, shuffle=True, num_workers=0这几个参数
# 然后遍历DataLoader，通过enumerate()函数将DataLoader中的数据和数据下标组合成一个索引序列，然后遍历这个索引序列
# 在遍历的过程中，将每个批次的数据和标签打印出来，当遍历到第3个批次的时候，就退出遍历
# 这样就完成了一个简单的数据集的生成和遍历的过程

以下将一个CSV文件作为数据源，然后通过自定义的Dataset和DateLoader读取数据

![image](./img/image.png)

In [14]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader

# 自定义 CSV 数据集，类似于上面的 MyDataset
class CSVDataset(Dataset):
    def __init__(self, file_path):
        # 读取 CSV 文件
        self.data = pd.read_csv(file_path)

    def __len__(self):
        # 返回数据集大小
        return len(self.data)

    def __getitem__(self, idx):
        # 使用 .iloc 明确基于位置索引
        row = self.data.iloc[idx]
        # 将特征和标签分开
        features = torch.tensor(row.iloc[:-1].to_numpy(), dtype=torch.float32)  # 特征
        label = torch.tensor(row.iloc[-1], dtype=torch.float32)  # 标签
        return features, label

# 实例化数据集和 DataLoader
dataset = CSVDataset("runoob_pytorch_data.csv")
dataloader = DataLoader(dataset, batch_size=5, shuffle=True)

# 遍历 DataLoader
for features, label in dataloader:
    print("特征:", features)
    print("标签:", label)
    break

特征: tensor([[ 1.2000,  2.1000, -3.0000],
        [ 1.5000,  2.2000, -1.1000],
        [-1.1000,  0.8000,  1.5000],
        [ 1.0000,  1.1000, -2.0000],
        [-0.3000,  0.8000,  1.2000]])
标签: tensor([1., 0., 1., 0., 0.])
